In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

### Instrumental Variables

In [2]:
hw_41 = pd.read_csv('homework_4.1.csv', index_col=0)
hw_41.head()

,Z,W,X,Y
0,0,-0.155644,-0.496971,0.282484
1,1,0.529539,2.284240,4.740596
2,1,0.910514,0.872232,3.449569
3,1,-0.705476,2.157260,3.002531
4,0,-0.590874,-0.386730,-1.848796


#### Finding effect without W

In [3]:
# Z = 0
Z_0 = hw_41[hw_41['Z'] == 0].copy()
# Z = 1
Z_1 = hw_41[hw_41['Z'] == 1].copy()

In [4]:
# Average of Y for Z = 1 and Z = 0
Y_1 = Z_1['Y'].mean()
Y_0 = Z_0['Y'].mean()
# Average of X for Z = 1 and Z = 0
X_1 = Z_1['X'].mean()
X_0 = Z_0['X'].mean()

In [5]:
# Finding effect by subtracting the averages
effect_1 = (Y_1 - Y_0) / (X_1 - X_0)
print(effect_1)

1.5618587073955674


#### Finding effect using W

In [6]:
print(hw_41.W.min())
print(hw_41.W.max())

-3.3034061894656137
4.783256692238601


In [7]:
hw_41_bin = hw_41[(hw_41.W > 2.5) & (hw_41.W < 4.5)].copy()
hw_41_bin

,Z,W,X,Y
122,0,2.602465,0.605367,3.681063
186,1,2.618520,0.586077,4.539243
255,1,2.827162,1.232351,4.663265
331,0,3.397541,-1.113033,2.033575
719,0,2.797805,-0.896273,1.299714
725,1,2.508496,-0.124078,4.678081
821,1,2.561691,1.913369,5.816817
1225,1,2.838006,0.127180,4.055567
1376,1,2.771416,2.115691,5.723919
1721,0,2.815178,-0.504554,1.715720


In [8]:
Y_diff = hw_41_bin[hw_41_bin.Z == 1].Y.mean() - hw_41_bin[hw_41_bin.Z == 0].Y.mean()
X_diff = hw_41_bin[hw_41_bin.Z == 1].X.mean() - hw_41_bin[hw_41_bin.Z == 0].X.mean()
effect_2 = Y_diff / X_diff
print(effect_2)

1.7222083220048707


### Regression Discontinuity

In [9]:
hw_42a = pd.read_csv('homework_4.2.a.csv', index_col=0)
hw_42b = pd.read_csv('homework_4.2.b.csv', index_col=0)
hw_42a.head()

,X,Y
0,81.822339,1
1,92.487870,0
2,85.372460,0
3,78.828025,0
4,75.807080,1


In [10]:
hw_42b.head()

,X2,Y2
0,76.643034,1
1,87.743397,1
2,81.639469,1
3,73.740485,0
4,90.480268,1


In [11]:
cutoff = 80.0
hw_42a['treated'] = (hw_42a.X >= cutoff).astype(int)
hw_42b['treated'] = (hw_42b.X2 >= cutoff).astype(int)
hw_42a['centered'] = hw_42a.X - cutoff
hw_42b['centered'] = hw_42b.X2 - cutoff

In [12]:
model_a = smf.ols('Y ~ centered * treated', data=hw_42a).fit()
model_b = smf.ols('Y2 ~ centered * treated', data=hw_42b).fit()
model_a.summary(), model_b.summary()

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      Y   R-squared:                       0.090
 Model:                            OLS   Adj. R-squared:                  0.090
 Method:                 Least Squares   F-statistic:                     3300.
 Date:                Sun, 15 Jun 2025   Prob (F-statistic):               0.00
 Time:                        11:33:04   Log-Likelihood:                -67412.
 No. Observations:              100000   AIC:                         1.348e+05
 Df Residuals:                   99996   BIC:                         1.349e+05
 Df Model:                           3                                         
 Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------